In [3]:
import torch
%load_ext autoreload
%autoreload 2

In [4]:
from model.threedepn import ThreeDEPNDecoder
from util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 263168  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 65792   
4  | bottleneck.3 | ReLU             | 0       
5  | bottleneck.4 | Linear           | 131584  
6  | bottleneck.5 | ReLU             | 0       
7  | decoder1     | Sequential       | 8389376 
8  | decoder1.0   | ConvTranspose3d  | 8388864 
9  | decoder1.1   | BatchNorm3d      | 512     
10 | decoder1.2   | ReLU             | 0       
11 | decoder2     | Sequential       | 2097536 
12 | decoder2.0   | ConvTranspose3d  | 2097280 
13 | decoder2.1   | BatchNorm3d      | 256     
14 | decoder2.2   | ReLU             | 0       
15 | decoder3     | Sequential       | 524480  
16 | decoder3.0   | ConvTranspose3d  | 524352  
17 | decoder3.1   | BatchNorm3d      | 128     
18 | decoder3.2   | ReLU          

In [5]:
from data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 153540
Length of overfit set: 64


In [14]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes

sample = train_dataset[5000]
print(f'Target DF: {sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(sample['target_df'], level=1)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [11]:
from training import train
config = {
    'experiment_name': 'test1',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 4,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': False,
    'kl_weight': 1,
    'resume_ckpt': None

}
train.main(config)

Using device: cuda:0
Training params: 3
[001/00004] train_loss: 0.016460 kl_loss: 0.015628 normal_loss: 0.000832
[002/00004] train_loss: 0.010964 kl_loss: 0.010147 normal_loss: 0.000817
[003/00004] train_loss: 0.008280 kl_loss: 0.007464 normal_loss: 0.000816


KeyboardInterrupt: 

In [ ]:
from training import train
config = {
    'experiment_name': 'overfit_test',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 2,
    'print_every_n': 1,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'test': True,
    'kl_weight': 1,
    'resume_ckpt': 'overfit_train'

}
train.main(config)

In [13]:
from util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist

# load model
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load(f"runs/{config['experiment_name']}/model_best.ckpt", map_location='cpu'))

# load latent codes and latent variances
latent_vectors = torch.load(f"runs/{config['experiment_name']}/latent_best.pt", map_location = 'cpu')
log_vars = torch.load(f"runs/{config['experiment_name']}/log_var_best.pt", map_location = 'cpu')

# Create distributions
Dist = dist.Normal(latent_vectors[100], torch.exp(log_vars[100]))

# Sample
x_vad = Dist.rsample().unsqueeze(0)

# Forward pass
output_meshes = model(x_vad)

# Visualize
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

Output()